In [1]:
import waterhealer as wh
import json

In [2]:
source = wh.from_kafka(
    ['testing'],
    {
        'bootstrap.servers': 'localhost:9095, localhost:9096',
        'group.id': 'group-test-2',
        'auto.offset.reset': 'latest',
    },
    debug = True
)

In [3]:
def json_loads(row):
    return (row[0], json.loads(row[1]))

def plus(row):
    no = row[1]['no']
    if no > 70:
        raise Exception('row is bigger than 12')
    return (row[0], no + 1)

In [4]:
outputs = []
source.map(json_loads).map(plus).map(wh.healing, source = source).sink(outputs.append)

Output()

In [5]:
source.start()

In [6]:
wh.auto_shutdown(source, got_error = True, graceful = 60)

In [7]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [8]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9095', 'localhost:9096'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [9]:
producer.bootstrap_connected()

True

In [10]:
from datetime import datetime
import json

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [11]:
import confluent_kafka as ck

source.consumer.committed([ck.TopicPartition('testing', 0)])[0].offset

472

topic: testing, partition: 0, offset: 472, data: b'{"no": 0, "datetime": "2020-04-22 15:30:06.753043"}'


In [12]:
import time

for count in range(14):
    data = {'no': count, 'datetime': str(datetime.now())}
    print(data)
    publish_message(
        producer, 'testing', 'streaming', json.dumps(data)
    )
    time.sleep(1.0)

topic: testing, partition: 0, offset: 473, data: b'{"no": 0, "datetime": "2020-04-22 15:30:34.000094"}'


{'no': 0, 'datetime': '2020-04-22 15:30:34.000094'}
{'no': 1, 'datetime': '2020-04-22 15:30:35.000646'}


topic: testing, partition: 1, offset: 436, data: b'{"no": 1, "datetime": "2020-04-22 15:30:35.000646"}'
topic: testing, partition: 2, offset: 447, data: b'{"no": 2, "datetime": "2020-04-22 15:30:36.001187"}'


{'no': 2, 'datetime': '2020-04-22 15:30:36.001187'}


topic: testing, partition: 3, offset: 412, data: b'{"no": 3, "datetime": "2020-04-22 15:30:37.001679"}'


...

{'no': 11, 'datetime': '2020-04-22 15:30:45.021241'}


topic: testing, partition: 2, offset: 448, data: b'{"no": 12, "datetime": "2020-04-22 15:30:46.026150"}'


{'no': 12, 'datetime': '2020-04-22 15:30:46.026150'}


topic: testing, partition: 3, offset: 413, data: b'{"no": 13, "datetime": "2020-04-22 15:30:47.029193"}'


{'no': 13, 'datetime': '2020-04-22 15:30:47.029193'}
